In [ ]:
!pip install langchain langchain-community openai tqdm


In [ ]:
!pip install -qU langchain-community

In [ ]:
!pip install groq

In [ ]:
!pip install langchain-groq


In [ ]:
!pip install crewai_tools

# **Import libraries**

In [ ]:
import os
from crewai import Crew, Agent, Task, Process
from langchain_groq import ChatGroq
from langchain_community.utilities.financial_datasets import FinancialDatasetsAPIWrapper
from langchain_community.agent_toolkits.financial_datasets.toolkit import FinancialDatasetsToolkit

In [ ]:
import os
from langchain_community.agent_toolkits.financial_datasets.toolkit import FinancialDatasetsToolkit
from langchain_community.utilities.financial_datasets import FinancialDatasetsAPIWrapper

#**API KEY**

In [ ]:
# Define la API key
os.environ["FINANCIAL_DATASETS_API_KEY"] = "8b969e06-4ac0-4785-b4b6-b174729b5d35"
os.environ["GROQ_API_KEY"] = "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"


# **GROQ: MODELS**

In [ ]:
from langchain_groq import ChatGroq
from langchain_core.tools import tool

# Inicializa el modelo de Groq
model1 = ChatGroq(model="groq/llama-3.3-70b-versatile",temperature=0.3,max_tokens=800)
model2 = ChatGroq(model="groq/deepseek-r1-distill-qwen-32b", temperature=0.4,max_tokens=1000)

# **Crew AI: Tools**

In [ ]:
# Inicializa el wrapper y el toolkit
api_wrapper = FinancialDatasetsAPIWrapper(
    financial_datasets_api_key=os.environ["FINANCIAL_DATASETS_API_KEY"]
)
toolkit = FinancialDatasetsToolkit(api_wrapper=api_wrapper)

In [ ]:
tools = toolkit.get_tools()



In [ ]:
from langchain_core.tools import Tool

# Helper para adaptar herramientas a CrewAI
def adapt_tools_for_crewai(tools):
    valid_tools = []
    for tool in tools:
        if isinstance(tool, Tool):
            valid_tools.append(tool)
        elif isinstance(tool, dict) and all(k in tool for k in ["name", "description", "func"]):
            valid_tools.append(tool)
    return valid_tools

# Adaptar las herramientas del toolkit
adapted_tools = adapt_tools_for_crewai(toolkit.get_tools())


In [ ]:
from google.colab import drive
import os


# Montar Drive
drive.mount('/content/drive')

# Definir ruta de la carpeta
ruta_carpeta = '/content/drive/MyDrive/sample'

# Mostrar archivos
archivos = os.listdir(ruta_carpeta)
print("Archivos en la carpeta:", archivos)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Archivos en la carpeta: ['TSLA.md', 'KO.md']


In [ ]:
from crewai_tools import DirectoryReadTool, FileReadTool, SerperDevTool

# Crear instancias correctamente (¡sin coma después de DirectoryReadTool!)
directory_read_tool = DirectoryReadTool(directory='/content/drive/MyDrive/sample')
file_read_tool = FileReadTool()
search_tool = SerperDevTool()

extra_tools = [directory_read_tool, file_read_tool, search_tool]


# **Crew AI: Agents**

In [ ]:
# Agent 1: Owner Earnings Valuator
owner_earnings_valuator = Agent(
    role="Owner Earnings Valuator",
    goal="Generate a concise summary of the company's intrinsic value based on the owner earnings method.",
    backstory=(
        "You are a financial analyst influenced by Warren Buffett’s thinking. "
        "You focus on real cash flows available to shareholders and explain your findings briefly and clearly "
        "for executive audiences with no unnecessary technical depth."
    ),
    tools=adapted_tools,
    llm=model1,
    verbose=True
)


# Agent 2: DCF Valuator
dcf_valuator = Agent(
    role="Discounted Cash Flow (DCF) Valuator",
    goal="Provide a high-level summary of the company's intrinsic value using the DCF method.",
    backstory=(
        "You are a valuation expert who communicates future cash flow projections in a simple, digestible way. "
        "Avoid going deep into formulas — focus on clarity and what matters for decision-making."
    ),
    tools=adapted_tools,
    llm=model1,
    verbose=True
)



investment_memo_editor = Agent(
    role="Executive Summary Editor",
    goal=(
        "Craft concise, structured Executive Summaries inspired by Warren Buffett’s memo style, "
        "based on valuation outputs and aligned with company samples."
    ),
    backstory=(
        "You are a seasoned financial communicator and value investor. "
        "You specialize in writing short, impactful executive memos. "
        "Your task is to synthesize valuation findings into a clear summary, using "
        "prior samples stored at '/content/drive/MyDrive/sample' to match tone, format, and depth. "
        "Start the memo with a short introduction: company name, business model, and customer persona. "
        "Avoid excessive detail—focus on clarity and investment logic."
    ),
    tools=extra_tools,
    llm=model2,
    verbose=True
)



# **Crew AI: Tasks**

In [ ]:
def define_tasks(ticker):
    owner_earnings_task = Task(
        description=f"Calculate the intrinsic value of {ticker} using the owner earnings method.",
        expected_output="Short summary Owner earnings-based valuation.",
        agent=owner_earnings_valuator
    )

    dcf_valuation_task = Task(
        description=f"Estimate the intrinsic value of {ticker} using the Discounted Cash Flow (DCF) method, do an analysis but dont go to dipped.",
        expected_output="DCF valuation short summary result.",
        agent=dcf_valuator
    )


    memo_editorial_task = Task(
    description=(
        f"Write a short executive summary for {ticker} using the tone, structure, and depth of previous memos stored in "
        f"'/content/drive/MyDrive/sample'. The memo must include:\n"
        f"1. A brief summary (2–3 lines) of the owner earnings valuation\n"
        f"2. A brief summary (2–3 lines) of the DCF valuation\n"
        f"3. A final investment recommendation in plain, confident language\n\n"
        f"The output should be clear, professional, and under 300 words. Match the formatting of prior reports. "
        f"Don't overexplain or go too technical. Prioritize clarity and investor logic."
    ),
    expected_output="Concise and well-formatted executive summary memo aligned with previous writing style.",
    agent=investment_memo_editor
)



    return [owner_earnings_task, dcf_valuation_task, memo_editorial_task]



# **Crew AI: Crews**

In [ ]:
def create_crew(tasks):
    crew = Crew(
        agents=[owner_earnings_valuator, dcf_valuator, investment_memo_editor],
        tasks=tasks,
        process=Process.sequential,
        verbose=True
    )
    return crew


# **Crew AI: Put all together**

In [ ]:
def generate_report(ticker):
    tasks = define_tasks(ticker)
    crew = create_crew(tasks)
    result = crew.kickoff()
    return result.output if hasattr(result, "output") else str(result)


In [ ]:
ticker = "AAPL"
report = generate_report(ticker)

with open(f"{ticker.lower()}_executive_report.md", "w", encoding="utf-8") as f:
    f.write(report)

print(f"✅ Reporte guardado como '{ticker.lower()}_executive_report.md'")



╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: 3623f305-a909-4cc8-8930-63561e20fb40                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

🚀 Crew: crew
└── 📋 Task: b552887e-3fcb-42c3-b5ef-76e26ef72cae
       Status: Executing Task...

🚀 Crew: crew
└── 📋 Task: b552887e-3fcb-42c3-b5ef-76e26ef72cae
       Status: Executing Task...
    └── 🤖 Agent: Owner Earnings Valuator
            Status: In Progress

# Agent: Owner Earnings Valuator
## Task: Calculate the intrinsic value of AAPL using the owner earnings method.


🚀 Crew: crew
└── 📋 Task: b552887e-3fcb-42c3-b5ef-76e26ef72cae
       Status: Executing Task...
    └── 🤖 Agent: Owner Earnings Valuator
            Status: In Progress
        └── 🧠 Thinking...

...

🚀 Crew: crew
├── 📋 Task: b552887e-3fcb-42c3-b5ef-76e26ef72cae
│      Assigned to: Owner Earnings Valuator
│      Status: ✅ Completed
│   └── 🤖 Agent: Owner Earnings Valuator
│           Status: ✅ Completed
├── 📋 Task: ae8369df-252c-41ad-809a-9f3bc7c2a711
│      Assigned to: Discounted Cash Flow (DCF) Valuator
│      Status: ✅ Completed
│   └── 🤖 Agent: Discounted Cash Flow (DCF) Valuator
│           Status: ✅ Completed
└── 📋 Task: dcec2aa5-db04-4409-928e-9ca04f9bf8ec
       Status: Executing Task...
    └── 🤖 Agent: Executive Summary Editor
            Status: ✅ Completed

🚀 Crew: crew
├── 📋 Task: b552887e-3fcb-42c3-b5ef-76e26ef72cae
│      Assigned to: Owner Earnings Valuator
│      Status: ✅ Completed
│   └── 🤖 Agent: Owner Earnings Valuator
│           Status: ✅ Completed
├── 📋 Task: ae8369df-252c-41ad-809a-9f3bc7c2a711
│      Assigned to: Discounted Cash Flow (DCF) Valuator
│      Status: ✅ Completed
│   └── 🤖 Agent: Discounted Cash Flow (DCF) Valuator
│           Status: ✅ Completed
└── 📋 Task: dcec2aa5-db04-4409-928e-9ca04f9bf8ec
       Assigned to: Executive Summary Editor
       Status: ✅ Completed
    └── 🤖 Agent: Executive Summary Editor
            Status: ✅ Completed

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: dcec2aa5-db04-4409-928e-9ca04f9bf8ec                                                                     │
│  Agent: Executive Summary Editor                                                                                │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name: crew                                                                                                     │
│  ID: 3623f305-a909-4cc8-8930-63561e20fb40                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

✅ Reporte guardado como 'aapl_executive_report.md'
